In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

from sklearn.cluster import KMeans
from sklearn.model_selection import cross_val_score
#from xgboost import XGBRegressor
from sklearn.preprocessing import LabelEncoder,OneHotEncoder
# load data
#from sklearn.model_selection import GridSearchCV
from sklearn.feature_selection import SelectKBest, f_classif, mutual_info_classif
import seaborn as sns
import matplotlib.pyplot as plt

### Read both test and train datasets

In [ ]:
train_ori = pd.read_csv('../input/tabular-playground-series-mar-2021/train.csv')
test_ori = pd.read_csv('../input/tabular-playground-series-mar-2021/test.csv')

## Explore train dataset
### Shape:

In [ ]:
print("Shape of data frame:" , train_ori.shape)
train_ori.head()

### Check for NA values :

In [ ]:
train_ori.info()

Data set is clean as far as NA are concerned. 

### Use .desscribe() method to look at different features

## Preprocessing :

1. dropping those categorical features that have more than 5 categories.


In [ ]:
sumry_obj = train_ori.describe(include='object')
print(sumry_obj)

In [ ]:
num_features = [
    "cont0", "cont1","cont2", "cont3","cont4", "cont5", "cont6", "cont7",
    "cont8", "cont9", "cont10"
]
obj_features = [
    "cat0", "cat1","cat2","cat3","cat4","cat6","cat9","cat11", "cat12", "cat13", "cat14", "cat15",
    "cat16", "cat17", "cat18"
]
target = train_ori["target"]

### Check for each categorical feature :
all unique categories in test and train dataset are identical  : It is found that they are.

good_cat_col = []
for colname in obj_features:
    diff_test_train = list(set(train_ori[colname].unique())& set(test_ori[colname].unique())).sort()
    if (diff_test_train == test_ori[colname].unique().sort()):
        good_cat_col.append(colname)  
        
len(good_cat_col)

### 2. Use label encoder for encoding all categorical variables

In [ ]:
#data_all = pd.concat([train_ori, test_ori])

In [ ]:
from sklearn import preprocessing
def lab_encoder (df, obj_features):
    le = preprocessing.LabelEncoder()
    obj_features_le =pd.DataFrame() 
    for colname in obj_features:
        obj_features_le[colname]=le.fit_transform(df[colname])

    return(obj_features_le)



In [ ]:
le_obj_col_train = lab_encoder(train_ori, obj_features)

le_obj_col_test = lab_encoder(test_ori, obj_features)

In [ ]:
def ohencoder (df):
    ohtrain = None

    for i in range (1,df.shape[1] ):
        col = df.iloc[:,i].values.reshape(df.shape[0],1)
        onehot_encoder = OneHotEncoder(sparse=False, categories='auto')
        feature = onehot_encoder.fit_transform(col)
        if ohtrain is None:
            ohtrain= feature
        else:
            ohtrain = np.concatenate((ohtrain, feature), axis=1)

    return(ohtrain)



In [ ]:
allobj_features = pd.concat([le_obj_col_train, le_obj_col_test])
allobj_features.shape

In [ ]:
allobj = ohencoder(allobj_features)

In [ ]:
oh_obj_train = allobj[:300000]
oh_obj_test = allobj[300000:]

In [ ]:
train_obj_oh = pd.DataFrame(oh_obj_train, index=train_ori.index)
test_obj_oh = pd.DataFrame(oh_obj_test, index=test_ori.index)

### 3. Remove outliers from numerical data

def remove_outliers(df, num_features):
    outliers_removed = []
    for colname in num_features:
        q25, q75 = np.percentile(df[colname], 25), np.percentile(df[colname], 75)
        iqr = q75 - q25
        cutoff = iqr*1.5
        lower,upper = q25-cutoff, q75+cutoff
        outliers = [x for x in train_ori['cont0'] if x > lower and x < upper]
        outliers_index = train_ori[train_ori['cont0'].isin(outliers)].index
        df= df.drop([outliers_index],inplace=True)
        outliers_removed.append(len(outliers))
        print('Outliers removed for column '+ str(colname) + 'is'+ str(outliers_removed))
    return(df,outliers_removed)


outliers_removed = []
q25, q75 = np.percentile(train_ori['cont0'], 25), np.percentile(train_ori['cont0'], 75)
iqr = q75 - q25
cutoff = iqr*1
lower,upper = q25-cutoff, q75+cutoff
outliers = [x for x in train_ori['cont0'] if x > lower and x < upper]
outliers_index = train_ori[train_ori['cont0'].isin(outliers)].index
#update_df= update_df.drop([outliers_index],inplace=True)
outliers_removed.append(len(outliers))
print('Outliers removed for column '+ str(colname) + 'is'+ str(outliers_removed))
#outliers_removed_index = [x for x in train_ori['cont0'] if x > lower and x < upper]

In [ ]:
def std_scaler(df,num_features):
    std_scaler = preprocessing.StandardScaler()
    std_num_col = std_scaler.fit_transform(df[num_features].values)
    scaled_num_feat_df = pd.DataFrame(std_num_col, index=df.index, columns=df[num_features].columns)
        
    return(scaled_num_feat_df)


In [ ]:
#for colname in obj_features:
 #   plt.figure() #this creates a new figure on which your plot will appear
  #  sns.histplot(data= train_ori, x= colname)


In [ ]:
#std_num_col_train = std_scaler(train_ori,num_features)
std_num_col_train = train_ori [num_features]
train_scaled_encoded = pd.concat ([std_num_col_train,train_obj_oh], axis=1)
train_scaled_encoded.head()
train_scaled_encoded.shape

### encode and scale test_ori dataframe :


In [ ]:
#std_num_col_test = std_scaler(test_ori,num_features)
std_num_col_test = test_ori [num_features]
test_scaled_encoded = pd.concat ([std_num_col_test,test_obj_oh], axis=1)
test_scaled_encoded.head()
test_scaled_encoded.shape

unique = sumry_obj.loc['unique',:] ## Row corresponding to freq 
unique_gt10= unique[unique>10].index ### names those cloumns that have fre more tha  2800
top_freq = sumry_obj.loc['top',freq_gt2800] ### row taht tells which 'level' was the most freqent
top_freq_gt2800 = top_freq[top_freq=='None'].index # names of those columns that have 'none' as most freq
data_all = data_all.drop(top_freq_gt2800 ,1) ## drop those columns
print('Shape of the dataframe now is: ', data_all.shape)

### f_classif_num, p_classif_num = f_classif(std_num_col_train, target)

p_classif_num


#electKBest(f_classif, k=5).fit(std_num_col_train, target)
#best_features_numeric= pd.DataFrame(best_features_numeric)

### Split X into X_train and X_valid, same for y

In [ ]:
from sklearn.model_selection import train_test_split
X_train, X_valid, y_train, y_valid = train_test_split(train_scaled_encoded, target, test_size=0.3, random_state=0)

In [ ]:
seed = 42

In [ ]:
### Random Forest classifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_absolute_error, roc_auc_score,accuracy_score

clf = RandomForestClassifier(n_estimators=700, max_depth=7, n_jobs=-1, random_state=seed)
clf.fit(X_train, y_train)
y_pred = clf.predict_proba(X_valid)[:, 1] # This grabs the positive class prediction
score = roc_auc_score(y_valid, y_pred)
print(f'{score:0.5f}') # 0.87323 shows we're doing better than a dummy model



In [ ]:
## LOgistics regression
from sklearn.linear_model import LogisticRegression

clf_log = LogisticRegression(penalty ='l1',solver='liblinear',max_iter=500,random_state=seed)
clf_log.fit(X_train, y_train)
y_pred = clf_log.predict_proba(X_valid)[:, 1] 
clf_log_score= clf_log.score(X_train, y_train)
score = roc_auc_score(y_valid, y_pred)
print(f'{score:0.5f}') 


In [ ]:
from sklearn.model_selection import cross_val_score
print(cross_val_score(clf_log, X_train, y_train, cv=10))


In [ ]:
from sklearn.neighbors import KNeighborsClassifier
neigh = KNeighborsClassifier(n_neighbors=5)
neigh.fit(X_train, y_train)
y_pred = neigh.predict_proba(X_valid)[:, 1] 
score = roc_auc_score(y_valid, y_pred)
print(f'{score:0.5f}') 

In [ ]:

from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
clf_xgb = XGBClassifier()
eval_set = [(X_valid, y_valid)]
clf_xgb.fit(X_train, y_train, early_stopping_rounds=10, eval_set = eval_set, eval_metric="logloss", verbose=True)
y_pred = clf_xgb.predict(X_valid) 
score = roc_auc_score(y_valid, y_pred)
print(f'{score:0.5f}') 
print(clf_xgb)





In [ ]:
clf_xgb.objective


In [ ]:
from sklearn.model_selection import cross_val_score
print(cross_val_score(clf_log, X_train, y_train, cv=10))

In [ ]:
from sklearn.model_selection import cross_val_score
print(cross_val_score(clf_xgb, X_train, y_train, cv=10))

In [ ]:
from sklearn.model_selection import cross_val_score
print(cross_val_score(neigh, X_train, y_train, cv=10))

GradientBoostingClassifier(criterion=’friedman_mse’, init=None,
 learning_rate=0.1, loss=’deviance’, max_depth=3,
 max_features=None, max_leaf_nodes=None,
 min_impurity_split=1e-07, min_samples_leaf=1,
 min_samples_split=2, min_weight_fraction_leaf=0.0,
 n_estimators=100, presort=’auto’, random_state=None,
 subsample=1.0, verbose=0, warm_start=False)

To DO :
1. Split train into X_train and X_valid
**data visulization**
correlation, MI score
feature selection

models :KNN, XGB DECISON TREE
3. K cluster 
4. xg BOOST , check rmse score


In [ ]:
# best model predictions 

best_model = clf_xgb

best_model.fit(train_scaled_encoded,target)

preds = best_model.predict(test_scaled_encoded)

In [ ]:
preds

In [ ]:

#score_dataset(label_X_train,y_train)
sub = pd.read_csv('../input/tabular-playground-series-mar-2021/sample_submission.csv')
sub['target']= preds
sub.to_csv('submission.csv', index=False)

In [ ]:
clf.fit(train_scaled_encoded, target)
submission['target'] = clf.predict_proba(test)[:, 1]
submission.to_csv('random_forest.csv')


In [ ]:
#score_dataset(label_X_valid,y_valid)

def hyperParameterTuning(X_train, y_train):
    param_tuning = {
        'learning_rate': [0.01, 0.1],
        'max_depth': [3, 5, 7, 10],
        #'min_child_weight': [1, 3, 5],
        #'subsample': [0.5, 0.7],
        #'colsample_bytree': [0.5, 0.7],
        'n_estimators' : [100, 200, 500],
        'objective': ['reg:squarederror']
    }

    xgb_model = XGBRegressor()

    gsearch = GridSearchCV(estimator = xgb_model,
                           param_grid = param_tuning,                        
                           #scoring = 'neg_mean_absolute_error', #MAE
                           scoring = 'neg_mean_squared_error',  #MSE
                           cv = 5,
                           n_jobs = -1,
                           verbose = 1)

    gsearch.fit(X_train,y_train)

    return gsearch.best_params_


best_parameters = hyperParameterTuning(label_X_train, y_train)